In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pkg_resources 
from tdev2.readers.gold_reader import *
from tdev2.readers.disc_reader import *

In [3]:
from tdev2.measures.ned import *
from tdev2.measures.boundary import *
from tdev2.measures.grouping import *
from tdev2.measures.coverage import *
from tdev2.measures.token_type import *

In [4]:
def prf2dict(dct, measurename, obj):
    # save precision-recall-f of tde objects to dictionary
    dct[measurename + '_P'] = obj.precision
    dct[measurename + '_R'] = obj.recall
    dct[measurename + '_F'] = obj.fscore
    
    return dct


def compute_scores(gold, disc, measures=[]):
    scores = dict()
    
    # Launch evaluation of each metric
    if len(measures) == 0 or "boundary" in measures:
        print('Computing Boundary...')
        boundary = Boundary(gold, disc)
        boundary.compute_boundary()
        scores = prf2dict(scores, 'boundary', boundary)
        
    if len(measures) == 0 or "grouping" in measures:
        print('Computing Grouping...')
        grouping = Grouping(disc)
        grouping.compute_grouping()
        scores = prf2dict(scores, 'grouping', grouping)    
        
    if len(measures) == 0 or "token/type" in measures:
        print('Computing Token and Type...')
        token_type = TokenType(gold, disc)
        token_type.compute_token_type()
        scores['token_P'],scores['token_R'],scores['token_F'] = token_type.precision[0], token_type.recall[0], token_type.fscore[0]
        scores['type_P'],scores['type_R'],scores['type_F'] = token_type.precision[1], token_type.recall[1], token_type.fscore[1]        
        
    if len(measures) == 0 or "coverage" in measures:
        print('Computing Coverage...')
        coverage = Coverage(gold, disc)
        coverage.compute_coverage()
        scores['coverage'] = coverage.coverage

        
    if len(measures) == 0 or "coverageNS" in measures:
        print('Computing Coverage No Single...')
        coverage = Coverage_NoSingleton(gold, disc)
        coverage.compute_coverage()
        scores['coverageNS'] = coverage.coverage

        
    if len(measures) == 0 or "ned" in measures:
        print('Computing NED...')
        ned = Ned(disc)
        ned.compute_ned()
        scores['ned'] = ned.ned
        
        
    
    return scores

In [6]:
import sys
sys.path.append('/home/korhan/Dropbox/tez_scripts/')
from utils.tde_utils import zrexp2tde

wrd_path = 'tdev2/share/phoenix.wrd'
phn_path = 'tdev2/share/phoenix.phn'

# exp_path = '/home/korhan/Desktop/zerospeech2017/track2/src/ZRTools/exp/'
exp_path = '/home/korhan/Desktop/tez/sdtw_exps/sdtw_A0_c3right_PCA_Wh_w10_L10_ext0.2_dia0.7_dOlapFalse_euclidLoss_ecut3/postZR_cost0.015_olap0.95_dedup0.5_edw0_dtw0/'

disc_clsfile = zrexp2tde(exp_path)

print(disc_clsfile)

/home/korhan/Desktop/tez/sdtw_exps/sdtw_A0_c3right_PCA_Wh_w10_L10_ext0.2_dia0.7_dOlapFalse_euclidLoss_ecut3/postZR_cost0.015_olap0.95_dedup0.5_edw0_dtw0/results/master_graph.class


In [82]:
gold = Gold(wrd_path=wrd_path, phn_path=phn_path) 
disc = Disc(disc_clsfile, gold) 



Discovered Class file read

4986 unique intervals, 1956 clusters with 6717 nodes found


In [8]:
disc_clsfile

'/home/korhan/Desktop/tez/sdtw_exps/sdtw_A0_c3right_PCA_Wh_w10_L10_ext0.2_dia0.7_dOlapFalse_euclidLoss_ecut3/postZR_cost0.015_olap0.95_dedup0.5_edw0_dtw0/results/master_graph.class'

In [9]:
scores = compute_scores(gold, disc)
scores['n_clus'] = len(disc.clusters)
scores['n_node'] = len(disc.intervals)
scores

Computing Boundary...
Computing Grouping...
Number of grouping jobs: 1
Computing Token and Type...
Computing Coverage...
Computing Coverage No Single...
Computing NED...


{'boundary_P': 0.3973320628870891,
 'boundary_R': 0.012563646771715224,
 'boundary_F': 0.024297842843555052,
 'grouping_P': 0.34531059683313,
 'grouping_R': 0.49911971830985913,
 'grouping_F': 0.4077245036845198,
 'token_P': 0.07741676694745286,
 'token_R': 0.006357991138343957,
 'token_F': 0.011750917089060384,
 'type_P': 0.27624309392265195,
 'type_R': 0.16246953696181965,
 'type_F': 0.20460358056265981,
 'coverage': 0.02069920951837666,
 'coverageNS': 0.02176615761444397,
 'ned': 0.7784343626890328,
 'n_clus': 1956,
 'n_node': 4986}